マルコフ連鎖を利用して、自動作文に挑戦する

markov.py

In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mkdir -p `mecab-config --dicdir`
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd
!apt-get -q -y install swig
!pip install mecab-python3

Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.58.0-2ubuntu3.5).
git is already the newest version (1:2.17.1-1ubuntu0.3).
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
  python-mecab sudo
0 upgraded, 13 newly installed, 0 to remove and 5 not upgraded.
Need to get 29.7 MB of archives.
After this operation, 284 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.1 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.1 [68.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 

In [2]:
!mecab

とっとこハム太郎
とっ	動詞,自立,*,*,五段・ラ行,連用タ接続,とる,トッ,トッ
とこ	動詞,非自立,*,*,五段・カ行イ音便,未然ウ接続,とく,トコ,トコ
ハム	名詞,一般,*,*,*,*,ハム,ハム,ハム
太郎	名詞,固有名詞,人名,名,*,*,太郎,タロウ,タロー
EOS
^C


In [3]:
import MeCab
import os, json, random

dict_file = "merkov_dict.json"
dic = {}

#辞書への登録
def regist_dic(wordlist):
  global dic
  w1 = ""
  w2 = ""
  
  #要素が3未満の場合は何もしない
  if len(wordlist) < 3: return
  
  for w in wordlist:
    word = w[0]
    if word == "" or word == "\r\n" or word == "\n": continue
    #辞書に単語を設定
    if w1 and w2:
      set_dic(dic, w1, w2, word)
    #文末を表す語の場合、連鎖をクリアする
    if word =="。" or word == "?" or word == "？":
      w1 = ""
      w2 = ""
      continue
    #次の前後関係を登録するために、単語をスライド
    w1, w2 = w2, word
    
  #辞書を保存
  json.dump(dic, open(dict_file, "w", encoding="utf-8"))
  
#応答文の作成
def set_dic(dic, w1, w2, w3):
  #新しい単語の場合は新しい辞書オブジェクトを作成
  if w1 not in dic: dic[w1] = {}
  if w2 not in dic[w1]: dic[w1][w2] = {}
  if w3 not in dic[w1][w2]: dic[w1][w2][w3] = 0
  #単語の出現集をインクリメントする
  dic[w1][w2][w3] += 1
  
#メイン処理
def make_resuponse(word):
  res = []
  
  #名詞、形容詞、動詞は文章の意図を表していることが多いと想定し、始点の単語とする
  w1 = word
  res.append(w1)
  w2 = word_choice(dic[w1])
  res.append(w2)
  while True:
    #w1, w2の組み合わせから予想される単語を選択
    if w1 in dic and w2 in dec[w1]: w3 = word_choice(dic[w1][w2])
    else: w3 = ""
    res.append(w3)
    #文末を表す語の場合、作文を終了
    if w3 == "。" or w3 =="?" or w3 == "？" or w3 == "": break
    #次の単語を選択するために単語をスライド
    w1, w2 = w2, w3
    
  return "".join(res)
  
def word_choice(candidate):
  keys = candidate.keys()
  return random.choice(list(keys))

#メイン処理

#辞書がすでに存在する場合は、最初に読み込む
if os.path.exists(dict_file):
  dic = json.load(open(dict_file, "r"))
  
while True:
  #標準入力から入力を受け付け、「さようなら」が入力されるまで続ける
  text = input("You -> ")
  if text == "" or text == "さようなら":
    print("Bot -> さようなら")
    break
    
  #文章整形
  if text[-1] != "。" and text[-1] != "?" and text != "？": text +="。"
    
  #形態素解析
  tagger = MeCab.Tagger()
  tagger.parse("")
  node = tagger.parseToNode(text)
  
  #マルコフ連鎖の辞書に登録
  wordlist = []
  while node is not None:
    hinshi = node.feature.split(".")[0]
    if hinshi not in ["BOS/EOS"]:
      wordlist.append([node.surface, hinshi])
    node = node.next
    
  #マルコフ連鎖の辞書に登録
  regist_dic(wordlist)
  
  #応答文の作成
  for w in wordlist:
    word = w[0]
    hinshi = w[1]
    #品詞が感動詞の場合は単語をそのまま返す
    if hinshi in ["感動詞"]:
      print("Bot -> " + word)
      break
    #品詞が名詞、形容詞、動詞の場合で、かつ、辞書に単語が存在する場合は作文して返す
    elif (hinshi in ["名詞", "形容詞", "動詞"]) and (word in dic):
      print("Bot ->" + make_response(word))
      break

You -> こんにちは
You -> 今日の夜ご飯は味噌汁がいいです
You -> 
Bot -> さようなら


できた！
